In [1]:
import requests
import bs4
import pandas as pd
from PIL import Image
import numpy as np
from tqdm import tqdm
from io import BytesIO

In [2]:
page = requests.get("https://pokemondb.net/pokedex/all")

In [3]:
parser = bs4.BeautifulSoup(page.content, "html.parser")

In [4]:
table = parser.find("table")

pokemon = []

for tr in table.find_all("tr")[1:]:

    cols = tr.find_all("td")
    info = cols[1].find_all(href = True)[0]
    
    types = cols[2].find_all('a', class_='type-icon')
    types = [i.get_text() for i in types]
    pokemon.append((info["href"], info.get_text(), types))
    
df = pd.DataFrame(pokemon, columns=["link","name", "pokemon_type"])
df["primary"] = df.pokemon_type.apply(lambda x: x[0])
df["secondary"] = df.pokemon_type.apply(lambda x: x[1] if len(x) > 1 else np.nan)
df = df.drop(columns=["pokemon_type"])

In [5]:
to_img_url = lambda x: x.replace("/pokedex/", "https://img.pokemondb.net/artwork/large/") + ".jpg"
df["img_url"] = df.link.apply(to_img_url)

In [6]:
for i in tqdm(range(len(df))):
    row = df.iloc[i]
    
    try: 
        response = requests.get(row.img_url, timeout=5)
        response.raise_for_status()

        img = Image.open(BytesIO(response.content))
        img.save(f"data/poke_images/{row.name}.png")
        
    except requests.HTTPError:
        pass

  0%|          | 0/1215 [00:00<?, ?it/s]

100%|██████████| 1215/1215 [05:31<00:00,  3.67it/s]


In [9]:
df["image_path"]  = df.index.to_series().apply(lambda x: f"data/poke_images/{x}.png")

import os
exists = df.image_path.apply(os.path.exists)
exists.mean()

1.0

In [10]:
df.to_csv("data/parsed_from_index.csv")